In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! rm -r temp
! mkdir temp
#! rm -r "/content/drive/My Drive/Minecraft/images"
#! rm -r "/content/drive/My Drive/Minecraft/checkpoint"
#! mkdir "/content/drive/My Drive/Minecraft/images"
#! mkdir "/content/drive/My Drive/Minecraft/checkpoint"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
rm: cannot remove 'temp': No such file or directory


In [ ]:
! pip install tensorflow-gpu==1.15
! pip install keras==2.1.3
! mkdir "/content/drive/My Drive/Minecraft/1"
! mkdir "/content/drive/My Drive/Minecraft/1/images"
! mkdir "/content/drive/My Drive/Minecraft/1/checkpoint"

     |████████████████████████████████| 411.5MB 42kB/s 
     |████████████████████████████████| 512kB 34.7MB/s 
     |████████████████████████████████| 3.8MB 37.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5ffba3976bfc1603fcf1fde6e5518c992e04b9d3486aadbb5d6539bceb7d839d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninst

In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, ZeroPadding3D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, UpSampling3D, Conv3D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
import os
from tensorboardcolab import *
tbc=TensorBoardColab()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' 
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.test.gpu_device_name())
import sys
import datetime
import numpy as np
from random import shuffle

with np.load("drive/My Drive/Minecraft/largeNorm.npz") as data:
    images = data['a']
print(tf.__version__)

shuffle(images)

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://9b29f5f6ac0b.ngrok.io
/device:GPU:0
1.15.0


In [ ]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 16
        self.img_depth = 16
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.img_depth, self.channels)
        self.latent_dim = 256

        optimizerD = Adam(0.0005)
        optimizerG = Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizerD,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizerG)
    
    def write_log(self, name, value, epoch):
        tbc.save_value("Loss", name, epoch, value)
        tbc.flush_line(name)
        tbc.close()

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 4 * 4 * 64, input_dim=self.latent_dim, activation="relu"))
        model.add(Reshape((64, 4, 4, 128)))
        model.add(UpSampling3D())
        model.add(Conv3D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling3D())
        model.add(Conv3D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv3D(1, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv3D(256, kernel_size=4, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv3D(128, kernel_size=4, strides=2, padding="same"))
        model.add(ZeroPadding3D(padding=((0,1),(0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv3D(64, kernel_size=4, strides=2, padding="same"))
        model.add(ZeroPadding3D(padding=((0,1),(0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv3D(32, kernel_size=4, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Conv3D(16, kernel_size=4, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv3D(8, kernel_size=4, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=16, save_interval=500):

        # Load the dataset
        X_train = images

        # Rescale -1 to 1
        #print(X_train[0][0])
        #X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=4)
        #print(X_train[0][0])

        # Adversarial ground truths
        valid = np.full((batch_size, 1), 0.9)
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            #epoch += 24000

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            #print(type(d_loss_real))
            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            #print ("%d [D Loss Real: %f, D Loss Fake: %f" % (epoch, d_loss_real, d_loss_fake))
            #print(d_loss_real)
            #print(d_loss_fake)
            self.write_log("g_loss", g_loss, epoch)
            self.write_log("d_loss", d_loss[0], epoch)
            self.write_log("Accuracy", d_loss[1]*100, epoch)
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
            if epoch % (2*save_interval) == 0:
                self.combined.save_weights('drive/My Drive/Minecraft/1/checkpoint/mnist-%d.h5' % epoch)
                #self.combined.save_weights('temp/mnist-%d.h5' % epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        #gen_imgs = 0.5 * gen_imgs + 0.5

        np.save("drive/My Drive/Minecraft/1/images/mnist_%d.npy" % epoch, gen_imgs)
        #np.save("temp/mnist_%d.npy" % epoch, gen_imgs)
        '''
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("C:/Users/adamj/Documents/MinecraftTerrainGeneration/DCGAN/images/mnist_%d.png" % epoch)
        plt.close()
        '''


In [ ]:
dcgan = DCGAN()
#dcgan.combined.load_weights("/content/drive/My Drive/Minecraft/checkpoint/mnist-24000.h5")
dcgan.train(epochs=100000, batch_size=16, save_interval=1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 128, 8, 8, 256)    16640     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128, 8, 8, 256)    0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 8, 8, 256)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 64, 4, 4, 128)     2097280   
_________________________________________________________________
zero_padding3d_3 (ZeroPaddin (None, 65, 5, 5, 128)     0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 65, 5, 5, 128)     512       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 65, 5, 5, 128)     0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.809229, acc.: 18.75%] [G loss: 0.864955]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 1.348994, acc.: 0.00%] [G loss: 0.476868]
2 [D loss: 0.783331, acc.: 6.25%] [G loss: 0.670638]
3 [D loss: 0.833014, acc.: 31.25%] [G loss: 0.845718]
4 [D loss: 0.714874, acc.: 34.38%] [G loss: 0.969436]
5 [D loss: 0.701949, acc.: 34.38%] [G loss: 1.088436]
6 [D loss: 0.665705, acc.: 34.38%] [G loss: 1.314684]
7 [D loss: 0.634192, acc.: 46.88%] [G loss: 1.137107]
8 [D loss: 0.667128, acc.: 37.50%] [G loss: 1.525097]
9 [D loss: 0.617384, acc.: 43.75%] [G loss: 1.679902]
10 [D loss: 0.450204, acc.: 46.88%] [G loss: 1.989478]
11 [D loss: 0.380173, acc.: 46.88%] [G loss: 1.902729]
12 [D loss: 0.468255, acc.: 37.50%] [G loss: 0.994180]
13 [D loss: 0.429614, acc.: 43.75%] [G loss: 0.393427]
14 [D loss: 0.364573, acc.: 46.88%] [G loss: 0.475755]
15 [D loss: 0.273194, acc.: 50.00%] [G loss: 0.382435]
16 [D loss: 0.270241, acc.: 50.00%] [G loss: 0.435992]
17 [D loss: 1.415858, acc.: 0.00%] [G loss: 0.405355]
18 [D loss: 1.100399, acc.: 0.00%] [G loss: 1.372739]
19 [D loss: 0.785695, a

In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()